# 0. Intro
1. 试图整合fixed和ascending两种拍卖
2. 直接输出LEN=300的data

# 1. Global settings
## 1.1 data path

In [6]:
LEN = 300
ARTIFICIAL = True  # 是否使用的人工数据集

# Small dataset
data_small_np_path = r'../../data/small_auctions_np.csv'

# Large data
data_large_np_path = r'E:\DATA\large_dta\large_auctions_np.csv'

# auction config
settings_NN_path = r"../data/target_datakey_all.csv"

# output path
data_path_root = "../../data/info_asymm/results/"
# target data is from method-2
if ARTIFICIAL:
    filename_head = "GT_1_artificial_LEN="
else:
    filename_head = "GT_1_LEN="
filename_tail = ".csv"

# 衡量一场auction是否unique的标志
unique_setting_GT = ['bidincrement','bidfee','retail','flg_endprice']
unique_setting_NN = ['desc','bidincrement','bidfee','retail','flg_endprice']


import numpy as np
#import cupy as np
import pandas as pd
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from visdom import Visdom

## 1.2 read

In [7]:
data_small = pd.read_csv(data_small_np_path, encoding="utf-8")
data_large = pd.read_csv(data_large_np_path, encoding="utf-8")
data = pd.concat([data_small,data_large],axis=0,ignore_index=True)

data_key = pd.read_csv(settings_NN_path, encoding="utf-8")

# 2. U&P
1. There is $n \leq T $
2. `U[i][0]` is initialized as 1 for convenience .
3. `U[i][j]` means：
> The probability that somebody makes the jth bid (given that j − 1 previous bids have been made)
4. `P` is the final output. `P[i]` represents the probability that auction ends at `duration=i+1`

In [8]:
col_names = []
tmp = np.arange(0,LEN)
tmp_str = [str(x) for x in tmp]
col_names.extend(tmp_str)

P_df = pd.DataFrame(columns=col_names)

P_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299


In [9]:
# For every uniq auction config i
for i in tqdm(range(0,data_key.shape[0])):

    # Get params
    v = float(data_key.loc[i,'retail'].item())            # retail price = valuation
    d = float(data_key.loc[i,'bidincrement'].item())      # bid increment
    b = float(data_key.loc[i,'bidfee'].item())            # bid fee

    # d==0 suggests a fixed-price auction.
    if d == 0:
        T = np.inf                                    # duration limitation
    else:
        T = np.floor((v-b)/d)                         # duration limitation

    # Solve for U with length of LEN
    U = [0] * (LEN + 2)                               # the prob. that someone offers a bid in t_th round
    U[0],U[1] = 1,1                                   # Actually we do not need u[0]. U[1]=1 ensures that the auction will last for more than 1 round.
    for t in range(2,len(U)):
        if(t<T):            # Not exceed the upper limit T. Caculable.
            U[t] = 1.0-(b/(v-d*(t-1)))
        else:               # Exceed the upper limit T. Not caculable and set as zero.
            U[t] = 0.0
        assert U[t]>=0, "U[t]<0 when t ={},and b = {},v = {}, d = {}".format(t,b,v,d)

    # Solve for P with length of LEN
    P = np.array([0.0]*(LEN+1))
    P[0] = 0.0                                            # Prob of auction duration==0, which is 0.
    tmp = np.array([0.0]*(LEN+3))
    tmp[0] = 1.0

    # Note：P[i][t] = U[i][1]*U[i][2]*...*(1-U[i][t+1])
    for t in range(1,len(P)):
        tmp[t] = tmp[t-1]*U[t]                          # tmp[t] is the continued product from U[1] to U[t].
        P[t] = (1-U[t+1])*tmp[t]

    # Dele the P[0]
    P = np.delete(P,[0],axis=0)
    assert len(P)==LEN,"P has wrong length (should be LEN)"

    pd_tmp = pd.DataFrame(P).T
    pd_tmp.columns = col_names
    P_df = pd.concat([P_df,pd_tmp],ignore_index=True)

print("Done")

  0%|          | 0/1276 [00:00<?, ?it/s]

Done


# 3. save

In [10]:
P_df.shape

(1276, 300)

In [11]:
filename_P = data_path_root + filename_head + str(LEN) + filename_tail
P_df.to_csv(filename_P,header=True,index=False,encoding="utf-8")
print(filename_P)
print("DONE")

../../data/info_asymm/results/GT_1_LEN=300.csv
DONE
